# Grid and Scenario Classes Cheat Sheet

In [1]:
import pandas as pd

from pprint import pprint
from powersimdata import Scenario

In [2]:
def classAttrs(obj, showContent=True):
    if showContent:
        return {attr: 'function' if callable(getattr(obj, attr)) 
                else getattr(obj, attr) for attr in dir(obj) if not attr.startswith("__")}
    else:
        return [attr for attr in dir(obj) if not attr.startswith("__")]

## Scenario Class
Here, We consider our *2020 base scenario*. This is the scenario presented in the scrolly telling on our webpage: https://science.breakthroughenergy.org/.

In [3]:
s = Scenario("824")

Initialized remote filesystem with ssh_fs,profile_fs,scenario_fs
Transferring ScenarioList.csv from ssh_fs


100%|##########| 669k/669k [00:01<00:00, 153kb/s]  


Transferring ScenarioList.csv.2 from scenario_fs


0.00b [00:00, ?b/s]

Transferring ExecuteList.csv from ssh_fs


100%|##########| 50.5k/50.5k [00:00<00:00, 76.7kb/s]


Transferring ExecuteList.csv.2 from scenario_fs
SCENARIO: Julia | USABase_2020_Anchor_profile_fix_1

--> State
analyze
--> Loading grid
Initialized remote filesystem with ssh_fs,profile_fs,scenario_fs
Transferring ScenarioList.csv from ssh_fs


100%|##########| 669k/669k [00:01<00:00, 153kb/s]  


Transferring ScenarioList.csv.2 from scenario_fs
Loading bus
Loading plant
Loading heat_rate_curve
Loading gencost_before
Loading gencost_after
Loading branch
Loading dcline
Loading sub
Loading bus2sub
--> Loading ct


In [4]:
pprint(classAttrs(s))

{'_default_info': [('plan', ''),
                   ('name', ''),
                   ('state', 'create'),
                   ('grid_model', ''),
                   ('interconnect', ''),
                   ('base_demand', ''),
                   ('base_hydro', ''),
                   ('base_solar', ''),
                   ('base_wind', ''),
                   ('change_table', ''),
                   ('start_date', ''),
                   ('end_date', ''),
                   ('interval', ''),
                   ('engine', '')],
 '_execute_list_manager': <powersimdata.data_access.execute_list.ExecuteListManager object at 0x11ff90640>,
 '_scenario_list_manager': <powersimdata.data_access.scenario_list.ScenarioListManager object at 0x11ff905e0>,
 '_set_info': 'function',
 '_set_status': 'function',
 '_setattr_allowlist': {'_execute_list_manager',
                        '_scenario_list_manager',
                        'data_access',
                        'info',
                        '

## Grid Class
* A load zone is always within a single interconnect
* A state may have multiple loadzones, not all are in the same interconnect
* Each substation has a unique location (GPS coordinates) and hence buses and plants attached to the same 
substation share the same location

In [5]:
grid = s.get_grid()

In [6]:
pprint(classAttrs(grid, showContent=False))

['SUPPORTED_ENGINES',
 'SUPPORTED_MODELS',
 'branch',
 'bus',
 'bus2sub',
 'data_loc',
 'dcline',
 'gencost',
 'grid_model',
 'id2zone',
 'interconnect',
 'model_immutables',
 'plant',
 'storage',
 'sub',
 'zone2id']


* Branch capacity is RateA
* A zero capacity means that the branch has effectively unlimited capacity and zero distance

In [7]:
print(grid.branch.dtypes)

from_bus_id             int64
to_bus_id               int64
r                     float64
x                     float64
b                     float64
rateA                 float64
rateB                 float64
rateC                 float64
ratio                 float64
angle                 float64
status                  int64
angmin                float64
angmax                float64
Pf                    float64
Qf                    float64
Pt                    float64
Qt                    float64
mu_Sf                 float64
mu_St                 float64
mu_angmin             float64
mu_angmax             float64
branch_device_type     object
interconnect           object
from_zone_id            int64
to_zone_id              int64
from_zone_name         object
to_zone_name           object
from_lat              float64
from_lon              float64
to_lat                float64
to_lon                float64
dtype: object


In [8]:
print(grid.bus.dtypes)

type              int64
Pd              float64
Qd              float64
Gs              float64
Bs              float64
zone_id           int64
Vm              float64
Va              float64
baseKV          float64
loss_zone         int64
Vmax            float64
Vmin            float64
lam_P           float64
lam_Q           float64
mu_Vmax         float64
mu_Vmin         float64
interconnect     object
lat             float64
lon             float64
dtype: object


DC line is a high voltage, long distance, direct current power lines

In [9]:
print(grid.dcline.dtypes) 

from_bus_id            int64
to_bus_id              int64
status                 int64
Pf                   float64
Pt                   float64
Qf                   float64
Qt                   float64
Vf                   float64
Vt                   float64
Pmin                 float64
Pmax                 float64
QminF                float64
QmaxF                float64
QminT                float64
QmaxT                float64
loss0                float64
loss1                float64
muPmin               float64
muPmax               float64
muQminF              float64
muQmaxF              float64
muQminT              float64
muQmaxT              float64
from_interconnect     object
to_interconnect       object
dtype: object


Generator cost curve specifies the cost as a function of power generated ($/MWh). These are defined in our model (`before`) and are eventually linearized in our simulation engine (`after`).

In [10]:
print(grid.gencost['before'].dtypes)

type              int64
startup         float64
shutdown        float64
n                 int64
c2              float64
c1              float64
c0              float64
interconnect     object
dtype: object


In [11]:
print(grid.gencost['after'].dtypes)

type              int64
startup         float64
shutdown        float64
n                 int64
c2              float64
c1              float64
c0              float64
p1              float64
f1              float64
p2              float64
f2              float64
interconnect     object
dtype: object


The plant data frame encloses the information on the generator in our model. `Pmax` is the maximum capacity of the generator.

In [12]:
print(grid.plant.dtypes)

bus_id            int64
Pg              float64
Qg              float64
Qmax            float64
Qmin            float64
Vg              float64
mBase           float64
status            int64
Pmax            float64
Pmin            float64
Pc1             float64
Pc2             float64
Qc1min          float64
Qc1max          float64
Qc2min          float64
Qc2max          float64
ramp_agc        float64
ramp_10         float64
ramp_30         float64
ramp_q          float64
apf             float64
mu_Pmax         float64
mu_Pmin         float64
mu_Qmax         float64
mu_Qmin         float64
type             object
GenFuelCost     float64
interconnect     object
zone_id           int64
zone_name        object
lat             float64
lon             float64
GenIOB          float64
GenIOC          float64
GenIOD          float64
dtype: object


Storage units can be added when we create a scenario. This is handled by the `ChangeTable` class. This scenario has no storage units.

In [13]:
pprint(grid.storage)

{'InEff': None,
 'LossFactor': None,
 'OutEff': None,
 'StorageData': Empty DataFrame
Columns: [UnitIdx, InitialStorage, InitialStorageLowerBound, InitialStorageUpperBound, InitialStorageCost, TerminalStoragePrice, MinStorageLevel, MaxStorageLevel, OutEff, InEff, LossFactor, rho, ExpectedTerminalStorageMax, ExpectedTerminalStorageMin]
Index: [],
 'duration': None,
 'energy_value': None,
 'gen': Empty DataFrame
Columns: [bus_id, Pg, Qg, Qmax, Qmin, Vg, mBase, status, Pmax, Pmin, Pc1, Pc2, Qc1min, Qc1max, Qc2min, Qc2max, ramp_agc, ramp_10, ramp_30, ramp_q, apf, mu_Pmax, mu_Pmin, mu_Qmax, mu_Qmin]
Index: []

[0 rows x 25 columns],
 'gencost': Empty DataFrame
Columns: [type, startup, shutdown, n, c2, c1, c0]
Index: [],
 'genfuel': [],
 'max_stor': None,
 'min_stor': None,
 'terminal_max': None,
 'terminal_min': None}


Below is the substation data frame.

In [14]:
print(grid.sub)

                           name  interconnect_sub_id        lat         lon  \
sub_id                                                                        
0                      AUBURN 1                    1  43.976100  -70.221100   
1                      AUBURN 2                    2  44.101000  -70.305300   
2                      AUBURN 3                    3  44.179900  -70.186000   
3                      AUBURN 4                    4  44.143300  -70.224100   
4                      DURHAM 1                    5  44.020700  -70.180500   
...                         ...                  ...        ...         ...   
41078           OREGON PORTLAND                81019  45.435890 -124.324466   
41079        WASHINGTON RAYMOND                81020  46.576647 -124.349185   
41080   WASHINGTON OCEAN SHORES                81021  46.979165 -124.478275   
41081         WASHINGTON QUEETS                81022  47.534671 -124.655429   
41082         WASHINGTON OZETTE                81023

## Scenario.state - Analyze class

In [15]:
pprint(classAttrs(s.state))

{'_data_access': <powersimdata.data_access.data_access.SSHDataAccess object at 0x10a61ddc0>,
 '_enter': 'function',
 '_execute_list_manager': <powersimdata.data_access.execute_list.ExecuteListManager object at 0x11ff90640>,
 '_get_data': 'function',
 '_leave': 'function',
 '_parse_infeasibilities': 'function',
 '_scenario': <powersimdata.scenario.scenario.Scenario object at 0x10a61dd60>,
 '_scenario_info': OrderedDict([('id', '824'),
                                ('plan', 'Julia'),
                                ('name', 'USABase_2020_Anchor_profile_fix_1'),
                                ('state', 'analyze'),
                                ('grid_model', 'usa_tamu'),
                                ('interconnect', 'USA'),
                                ('base_demand', 'vJan2021'),
                                ('base_hydro', 'vJan2021'),
                                ('base_solar', 'vJan2021'),
                                ('base_wind', 'vJan2021'),
                     

                            11782: 0,
                            11783: 0,
                            11801: 0,
                            11802: 0,
                            11803: 0,
                            11806: 0,
                            11836: 0,
                            11862: 0,
                            11867: 0,
                            11868: 0,
                            11869: 0,
                            11870: 0,
                            11871: 0,
                            11872: 0,
                            11879: 0,
                            12030: 0,
                            12031: 0,
                            12032: 0,
                            12033: 0,
                            12227: 0,
                            12228: 0,
                            13033: 0,
                            13266: 0,
                            13271: 0,
                            13272: 0},
               'zone_id': {1: 0.9849350329220252,

                              52: 1.0,
                              201: 0.04051051349101375,
                              202: 0.8319519856828816,
                              203: 1.061072929156075,
                              204: 1.061072929156075,
                              205: 1.061072929156075,
                              206: 1.061072929156075,
                              207: 1.061072929156075,
                              208: 1.361845748310557,
                              209: 1.4270784267545928,
                              210: 6.485148514851485,
                              211: 2.67497034400949,
                              212: 3.300055463117027,
                              213: 1.0,
                              214: 1.6384563303994581,
                              216: 1.4285714285714286,
                              301: 0.9722222222222222,
                              303: 2.57183908045977,
                              306: 1.442703232125367

Branch congestion:
* CONGU and CONGL give the congestion upper and lower flow limits on a line
* A branch may be uncongested or congested in one direction
* A branch is never congested in both directions at the same time

In [16]:
congl = s.get_congl()   # congestion lower limit, time x branch in $/MWh
congu = s.get_congu()   # congestion upper limit, time x branch in $/MWh

demand = s.get_demand() # demand, time x loadzone in MWh
lmp = s.get_lmp()       # locational marginal price, time x busId in $/MWh
pf = s.get_pf()         # power flow, time x branch in MWh
pg = s.get_pg()         # power generated, time x plant in MWh

--> Loading CONGL
--> Loading CONGU
--> Loading demand
Multiply demand in Maine (#1) by 0.98
Multiply demand in New Hampshire (#2) by 0.98
Multiply demand in Vermont (#3) by 0.98
Multiply demand in Massachusetts (#4) by 0.98
Multiply demand in Rhode Island (#5) by 0.98
Multiply demand in Connecticut (#6) by 0.98
Multiply demand in New York City (#7) by 0.99
Multiply demand in Upstate New York (#8) by 0.99
Multiply demand in New Jersey (#9) by 1.04
Multiply demand in Pennsylvania Eastern (#10) by 1.02
Multiply demand in Pennsylvania Western (#11) by 1.02
Multiply demand in Delaware (#12) by 1.04
Multiply demand in Maryland (#13) by 1.01
Multiply demand in Virginia Mountains (#14) by 1.06
Multiply demand in Virginia Tidewater (#15) by 1.06
Multiply demand in North Carolina (#16) by 1.04
Multiply demand in Western North Carolina (#17) by 1.04
Multiply demand in South Carolina (#18) by 1.04
Multiply demand in Georgia North (#19) by 1.01
Multiply demand in Georgia South (#20) by 1.01
Multip

These are our renewable resource profiles
* Meteoroligical data are used to obtain the solar irradiance and the wind speed at a solar plant and wind farm, respectively - Models are then used to derive the power output
* Because renewables are unpredictable, the energy available is often greater than the energy used
* Hydro is an exception because it's generally very stable day-to-day
* The waste of renewable energy is called "curtailment"
* Curtailment = energy_available - power_generated

In [17]:
hydro = s.get_hydro() # hydro profile, time x plant in MWh
solar = s.get_solar() # solar profile, time x plant in MWh
wind = s.get_wind()   # wind profile, time x plant in MWh

--> Loading hydro
--> Loading solar
--> Loading wind


In [18]:
print(hydro)

                        5         6         7         8         9      \
UTC                                                                     
2016-01-01 00:00:00  0.478794  0.478794  0.478794  0.478794  0.478794   
2016-01-01 01:00:00  0.476794  0.476794  0.476794  0.476794  0.476794   
2016-01-01 02:00:00  0.500844  0.500844  0.500844  0.500844  0.500844   
2016-01-01 03:00:00  0.513545  0.513545  0.513545  0.513545  0.513545   
2016-01-01 04:00:00  0.535973  0.535973  0.535973  0.535973  0.535973   
...                       ...       ...       ...       ...       ...   
2016-12-31 19:00:00  0.398887  0.398887  0.398887  0.398887  0.398887   
2016-12-31 20:00:00  0.371862  0.371862  0.371862  0.371862  0.371862   
2016-12-31 21:00:00  0.340430  0.340430  0.340430  0.340430  0.340430   
2016-12-31 22:00:00  0.340107  0.340107  0.340107  0.340107  0.340107   
2016-12-31 23:00:00  0.338237  0.338237  0.338237  0.338237  0.338237   

                        10        12        13    